In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2020 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [6]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\rony\AppData\Local\Temp\ipykernel_4992\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [7]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],NaN
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],NaN
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],NaN
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],NaN
...,...,...,...,...,...,...,...
269,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios,Robert Rodriguez (director/screenplay); Priyan...,NaN,[241]
270,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN,[242]
271,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN,[243]
272,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN,[244]


In [27]:
df_2020 = df[['Title','Cast and crew']]

In [9]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [10]:
!pip install tmdbv3api

You should consider upgrading via the 'c:\users\rony\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [11]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '22e607d16a8f50e45d9dbee11b0dd0b3'

In [12]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [28]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

IndexError: list index out of range

In [16]:
df_2020

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery
...,...,...,...
150,Top Gun: Maverick,"Joseph Kosinski (director); Ehren Kruger, Eric...",Action Drama
151,The Croods 2,"Joel Crawford (director); Kevin Hageman, Dan H...",Animation Adventure Family
152,Respect,Liesl Tommy (director); Tracey Scott Wilson (s...,Music Drama
153,The Last Duel,"Ridley Scott (director); Ben Affleck, Matt Dam...",Drama


In [15]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [16]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\rony\AppData\Local\Temp\ipykernel_4992\3848435694.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))


In [17]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [18]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\rony\AppData\Local\Temp\ipykernel_4992\3646981363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [19]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [20]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\rony\AppData\Local\Temp\ipykernel_4992\463054939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:

df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))

C:\Users\rony\AppData\Local\Temp\ipykernel_4992\2966500223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [23]:
df_2020

,Title,Cast and crew,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...
269,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
270,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
271,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
272,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [39]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [40]:
new_df20 = new_df20.rename(columns={'director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title'})

NameError: name 'new_df20' is not defined

In [31]:
new_df20

NameError: name 'new_df20' is not defined

In [32]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

NameError: name 'new_df20' is not defined

In [33]:
new_df20 = new_df20.dropna(how='any')

NameError: name 'new_df20' is not defined

In [34]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

NameError: name 'new_df20' is not defined

In [35]:
new_df20

NameError: name 'new_df20' is not defined

In [34]:
old_df = pd.read_csv('../input/final_data.csv')

In [35]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,toy story,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,grumpier old men,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,waiting to exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,father of the bride part ii,Steve Martin Diane Keaton Martin Short Charles...
...,...,...,...,...,...,...,...
36841,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
36842,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
36843,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
36844,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [36]:
final_df = old_df.append(new_df20,ignore_index=True)

In [37]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Animation Comedy Family,toy story,Tom Hanks Tim Allen Don Rickles John Lasseter ...
1,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Adventure Fantasy Family,jumanji,Robin Williams Jonathan Hyde Kirsten Dunst Joe...
2,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Romance Comedy,grumpier old men,Walter Matthau Jack Lemmon Ann-Margret Howard ...
3,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Comedy Drama Romance,waiting to exhale,Whitney Houston Angela Bassett Loretta Devine ...
4,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Comedy,father of the bride part ii,Steve Martin Diane Keaton Martin Short Charles...
...,...,...,...,...,...,...,...
36982,Joseph Kosinski,Tom Cruise,Miles Teller,Jennifer Connelly,Action Drama,top gun: maverick,Tom Cruise Miles Teller Jennifer Connelly Jose...
36983,Joel Crawford,Nicolas Cage,Emma Stone,Ryan Reynolds,Animation Adventure Family,the croods 2,Nicolas Cage Emma Stone Ryan Reynolds Joel Cra...
36984,Liesl Tommy,Jennifer Hudson,Forest Whitaker,Marlon Wayans,Music Drama,respect,Jennifer Hudson Forest Whitaker Marlon Wayans ...
36985,Ridley Scott,Matt Damon,Adam Driver,Jodie Comer,Drama,the last duel,Matt Damon Adam Driver Jodie Comer Ridley Scot...


In [ ]:
final_df.to_csv('main_data.csv',index=False)